In [11]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

import numpy as np

In [20]:
def pascal(nrows):
    a = [1]
    yield(a)
    
    for i in range(nrows):
        a = np.convolve(a, [1,1])
        yield(a)

In [21]:
for row in pascal(5):
    print(row)

[1]
[1 1]
[1 2 1]
[1 3 3 1]
[1 4 6 4 1]
[ 1  5 10 10  5  1]


In [22]:
row

array([ 1,  5, 10, 10,  5,  1])

In [23]:
ind = range(len(row))

range(0, 6)

In [24]:
from itertools import combinations

In [99]:
def all_combos(s):
    n = len(s)
    for r in range(1, n+1):
        for comb in combinations(s, r):
            yield list(comb)

In [100]:
row = np.array([1, 2, 1])
ind = set(range(n))

{0, 1, 2}

In [101]:
for comb in all_combos(ind):
    print(comb)

[0]
[1]
[2]
[0, 1]
[0, 2]
[1, 2]
[0, 1, 2]


In [110]:
def all_partitions(s):
    for numer in all_combos(s):
        rest = s - set(numer)
        for denom in all_combos(rest):
            yield numer, denom

In [111]:
for numer, denom in all_partitions(ind):
    print(numer, denom, row[numer].sum(), row[denom].sum())

[0] [1] 1 5
[0] [2] 1 10
[0] [3] 1 10
[0] [4] 1 5
[0] [5] 1 1
[0] [1, 2] 1 15
[0] [1, 3] 1 15
[0] [1, 4] 1 10
[0] [1, 5] 1 6
[0] [2, 3] 1 20
[0] [2, 4] 1 15
[0] [2, 5] 1 11
[0] [3, 4] 1 15
[0] [3, 5] 1 11
[0] [4, 5] 1 6
[0] [1, 2, 3] 1 25
[0] [1, 2, 4] 1 20
[0] [1, 2, 5] 1 16
[0] [1, 3, 4] 1 20
[0] [1, 3, 5] 1 16
[0] [1, 4, 5] 1 11
[0] [2, 3, 4] 1 25
[0] [2, 3, 5] 1 21
[0] [2, 4, 5] 1 16
[0] [3, 4, 5] 1 16
[0] [1, 2, 3, 4] 1 30
[0] [1, 2, 3, 5] 1 26
[0] [1, 2, 4, 5] 1 21
[0] [1, 3, 4, 5] 1 21
[0] [2, 3, 4, 5] 1 26
[0] [1, 2, 3, 4, 5] 1 31
[1] [0] 5 1
[1] [2] 5 10
[1] [3] 5 10
[1] [4] 5 5
[1] [5] 5 1
[1] [0, 2] 5 11
[1] [0, 3] 5 11
[1] [0, 4] 5 6
[1] [0, 5] 5 2
[1] [2, 3] 5 20
[1] [2, 4] 5 15
[1] [2, 5] 5 11
[1] [3, 4] 5 15
[1] [3, 5] 5 11
[1] [4, 5] 5 6
[1] [0, 2, 3] 5 21
[1] [0, 2, 4] 5 16
[1] [0, 2, 5] 5 12
[1] [0, 3, 4] 5 16
[1] [0, 3, 5] 5 12
[1] [0, 4, 5] 5 7
[1] [2, 3, 4] 5 25
[1] [2, 3, 5] 5 21
[1] [2, 4, 5] 5 16
[1] [3, 4, 5] 5 16
[1] [0, 2, 3, 4] 5 26
[1] [0, 2, 3, 5] 5 22
[1]

In [157]:
from fractions import Fraction

def enumerate_rows(n):
    for i, row in enumerate(pascal(n)):
        index = set(range(len(row)))
        for win, lose in all_partitions(index):
            numer, denom = row[win].sum(), row[lose].sum()
            eff = (numer + denom) / 2**i
            frac = Fraction(numer, denom)
            yield eff, -i, win, lose, frac.numerator, frac.denominator

In [158]:
for result in enumerate_rows(3):
    print(result)

(1.0, -1, [0], [1], 1, 1)
(1.0, -1, [1], [0], 1, 1)
(0.75, -2, [0], [1], 1, 2)
(0.5, -2, [0], [2], 1, 1)
(1.0, -2, [0], [1, 2], 1, 3)
(0.75, -2, [1], [0], 2, 1)
(0.75, -2, [1], [2], 2, 1)
(1.0, -2, [1], [0, 2], 1, 1)
(0.5, -2, [2], [0], 1, 1)
(0.75, -2, [2], [1], 1, 2)
(1.0, -2, [2], [0, 1], 1, 3)
(1.0, -2, [0, 1], [2], 3, 1)
(1.0, -2, [0, 2], [1], 1, 1)
(1.0, -2, [1, 2], [0], 3, 1)
(0.5, -3, [0], [1], 1, 3)
(0.5, -3, [0], [2], 1, 3)
(0.25, -3, [0], [3], 1, 1)
(0.875, -3, [0], [1, 2], 1, 6)
(0.625, -3, [0], [1, 3], 1, 4)
(0.625, -3, [0], [2, 3], 1, 4)
(1.0, -3, [0], [1, 2, 3], 1, 7)
(0.5, -3, [1], [0], 3, 1)
(0.75, -3, [1], [2], 1, 1)
(0.5, -3, [1], [3], 3, 1)
(0.875, -3, [1], [0, 2], 3, 4)
(0.625, -3, [1], [0, 3], 3, 2)
(0.875, -3, [1], [2, 3], 3, 4)
(1.0, -3, [1], [0, 2, 3], 3, 5)
(0.5, -3, [2], [0], 3, 1)
(0.75, -3, [2], [1], 1, 1)
(0.5, -3, [2], [3], 3, 1)
(0.875, -3, [2], [0, 1], 3, 4)
(0.625, -3, [2], [0, 3], 3, 2)
(0.875, -3, [2], [1, 3], 3, 4)
(1.0, -3, [2], [0, 1, 3], 3, 5)
(0

In [173]:
best = {}
absent = (-np.inf,)

for result in enumerate_rows(10):
    eff, i, win, lose, numer, denom = result
    key = numer, denom
    t = best.get(key, absent)
    if result > t:
        best[key] = result

In [174]:
for odds in range(1, 11):
    result = best[1, odds]
    eff, i, win, lose, numer, denom = result
    print(odds, -i, eff, win, lose)

1 1 1.0 [1] [0]
2 10 0.9990234375 [6, 7, 9, 10] [1, 2, 3, 4, 5, 8]
3 2 1.0 [2] [0, 1]
4 9 0.99609375 [1, 6, 8] [2, 3, 4, 5, 7]
5 7 0.984375 [5] [1, 2, 3, 4, 6]
6 9 0.998046875 [2, 7, 9] [1, 3, 4, 5, 6, 8]
7 3 1.0 [3] [0, 1, 2]
8 6 0.984375 [5, 6] [1, 2, 3, 4]
9 10 0.634765625 [1, 8, 9] [2, 4, 6, 7]
10 10 0.708984375 [1, 8, 9, 10] [3, 4, 6, 7]
